Amadéo David (44761700) - Marine Van Renterghem (31621700) - Sylvie Van Schendel (44841700)

# LINMA1702 Optimisation d'un réseau de distribution d'eau

## I. Analyse d'un réseau existant

### I.1 Que représentent les expressions $A^Th$ et $Af$ ?

Soit h le vecteur contenant les hauteurs en chaque point, f le vecteur contenant les débits circulant dans les conduites et A la matrice d'incidence.

L'expression $A^Th$ est le vecteur contenant les dénivelés de chaque conduite.

L'expression $Af$ est le vecteur contenant les débits nets en chaque point.

### I.2 

### I.4 Résolution numérique du problème I.3 avec les données fournies

In [3]:
from scipy.optimize import linprog # import du solveur d'optimisation linéaire à utiliser
import matplotlib.pyplot as plt # pour graphiques éventuels
import numpy as np

In [17]:
#Fonction retournant [f,xC,xA] avec f la valeur de la fonction objectif maximale 
#xC un vecteur des valeurs optimales des débits aux points de consommation
#xA un vecteur des valeurs optimales des débits aux points d'approvisionnement
#Soit Pf le prix facturé
#Soit C le cout
#Soit DCmin et DCmax les valeurs minimales et maximales du débit aux points de consommation
#Soit DAmax les valeurs maximales du débit extractibles aux sources
def Analyse(C,Pf,Dcmin,Dcmax,Damax):
    """
    Analyse renvoie un vecteur retournant, dans l'ordre :
        ° la valeur de la fonction objectif maximale;
        ° un vecteur des valeurs optimales des débits aux points de consommation;
        ° un vecteur des valeurs optimales des débits aux points d'approvisionnement.
        
    """
    c  = -1 * np.concatenate((Pf, -1*np.array(C)))       # On traite un problème de maximisation
    A1 = np.eye(len(Dcmin)+len(Damax))
    A2 = -1 * np.eye(len(Dcmin))                         # Matrice identite
    A2Add = np.zeros((len(Dcmin),len(Damax)))            # Matrice de 0
    A2 = np.concatenate((A2,A2Add),axis=1)               # Ajout de 0 a la fin de la matrice
    A  = np.concatenate((A1,A2))
    A3 = np.ones((1,len(Dcmin)+len(Damax)))
    A3[0,len(Dcmin):(len(Dcmin)+len(Damax))]*=-1
    A  = np.concatenate((A,A3))
    b  = np.concatenate((np.concatenate((Dcmax,Damax)),np.concatenate((-1*np.array(Dcmin),[0]))))
    res= linprog(c, A_ub = A, b_ub = b)
    print(c)
    return [-1*res.fun, res.x[0:len(Dcmin)],res.x[len(Dcmin):len(Dcmin)+len(Damax)]]

In [2]:
def readFile(fileName): 
    with open(fileName,"r") as f :
        nPoints = int(f.readline().split()[0])
        nConduites=int(f.readline().split()[0])
        f.readline()
        f.readline()
        xyz   = np.array(list(list(float(w) for w in f.readline().split()[:]) for i in range(nPoints)))
        f.readline()
        f.readline()
        A = np.array(list(list(float(w) for w in f.readline().split()[:]) for i in range(nPoints)))
        alpha=10**6
        R=1
        Pa=[2, 6]
        Damax=[10000, 5000]
        C= [0.2, 0.3]
        Pc=[1, 4, 10, 20, 23]
        Dcmin=[200, 200, 200, 200, 200]
        Dcmax=[4000, 2000, 2000, 3000, 2000]
        Pf=0.9
        Pi=[3, 5, 7, 8, 9, 11, 12 ,13, 14, 15, 16, 17, 18, 19, 21, 22]
        X = xyz[:,0]
        Y = xyz[:,1]
        Z = xyz[:,2]
        
    return [nPoints,nConduites,X,Y,Z,A,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf,Pi,alpha]
    

In [18]:
[nPoints,nConduites,X,Y,Z,A,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf,Pi,alpha]=readFile("Mini-exemple-avec-solution/Data-mini-exemple-avec-solution.txt")
print(Analyse(C,Pf*np.ones(len(Dcmin)),Dcmin,Dcmax,Damax))

[-0.9 -0.9 -0.9 -0.9 -0.9  0.2  0.3]
[8800.0, array([4000., 2000., 2000., 3000., 2000.]), array([10000.,  3000.])]


# II. Améliorations du réseau

# III. Conception d'un réseau optimal